# Man-made Structures Detection from Space

Authors: *Eduard Ribas Fernández*, *Peter Weber*

Advisor: *Jordi Vitrià* (Universidad de Barcelona), *Marco Bressan* (Satellogic)

Fundamentals of Data Science Master's Thesis - Universitat de Barcelona

July 2019

## 1. Introduction

### Motivation
Human kind exerts an ever increasing pressure on natural and ecological systems due to the associated consequences of the explosion of human population. The exploitation of the earth manifests itself in extraction of natural ressources, proliferation of human-made infrastructure and waste, and increasing production land use for crop and pastry land \parencite{kareiva2007}. As a logical consequence, we observe widespread declines in biodiversity \parencite{newbold2015}, 
decrease in natural habitat, attrition of wilderness areas, deforestation, and enhanced emission of greenhouse gases to the atmosphere. This increasing intrusion leads to reduction of benefits that humans receive from natural systems \parencite{costanza2014} such as the extinction of natural ressources, and ultimately provoke natural disaster induced by effects such as climate change. 

An essential prerequisite to mitigate human threat to nature is the access to data that allows for spatial and temporal mapping of human activity \parencite{raiter2014}. To this end, the last decades have brought about developments of affordable and recurrent remote sensing technology \parencite{hansen2013}. In particular, we now have public and continuous access to overhead imagery data for earth observation in different levels of detail, ranging from 100m to 0.01m. Overhead imagery data is obtained either by satellites or by airborne sensor systems. Additionally, remote sensing technologies open up the road for applications in agricultue, disaster recovery, urban development, and environmental mapping.

The task of detecting various types of man-made structure and man-induced change has become a key problem in remote sensing image analysis. However, unlike the computer vision community that disposes of datasets with thousands or millions of images containing up to thousands of distinct annotations \parencite{everingham2010, deng2009, lin2014, krasin2016}, the remote sensing community is only recently making first steps towards creating standardized labeled large scale datasets. Several approaches into this direction have been focussing either on classifying land cover and land use \parencite{sumbul2019} or annotating overhead images with object categories \parencite{vanetten2018, lam2018}. These annotations were used to perform object detection or segmentation \parencite{yang2010, krasin2016}, and to e.g. map roads and buildings \parencite{vanetten2018, vanetten2019}. However, the statistics of the images and categories in these works is heavily biased towards man-made structures so that wilderness areas are strongly underrepresented. 

The computer vision community has largely benefited from recent advances in deep learning ultimately leading to the outsourcing of convolutional neural networks pretrained on massive datasets. In the remote sensing community, researchers are recently also starting to follow this pathway \parencite{sumbul2019}. However, pretrained models are not yet widely available, so that many works in the remote sensing field are based on fine-tuning neural networks  pre-trained on traditional computer vision tasks.

### Satellogic
This work has been developed in cooperation with Satellogic, a company that provides earth observation data and analytics as a service to enable better decision making for industries, governments, and individuals. Satellogic was founded in 2010 in Buenos Aires, and has expanded since then with offices in Barcelona and China. Satellogic builds, launches and maintains their own satellites.

Satellogic focusses on developing heavily weight and cost optimized satellites. Their first nano satellite, called Capitán Beto, was sent to space in 2013 \parencite{wiki_satellogic}. Currently, Satellogic has 31 satellites orbiting earth, whose weight is 35kg, a minuscul fraction of conventional satellite systems (more than 1000kg \parencite{satellogic_youtube}). The satellites feature hyperspectral image aquisition at 1m pixel resolution. Satellogic envisions to have 300 satellites orbiting the earth within a few years providing real time imagery for any geospatial location. 

The hyperspectral technology i.e. image aquisition capability in more than 30 spectral bands allows for monitoring the earth with great detail \parencite{satellogic2019}. Every object and every plant has its own spectral fingerprint. Measuring the optical reflectance to the solar radiation for instance allows to distinguish between different kinds of crops, and its status of irrigation and fertilization. Further, it is possible to measure the level of pollution in the air and monitor vegetation below the water surface. Satellogic's clients apply this technology to map land use, monitor infrastructure, track agricultural development, evaluate the health of crops, and evaluate productivity of natural ressources.

At 1m pixel resolution, 10 satellites can remap 1 million square kilometers every 6 weeks. Note that the surface of the earth is roughly 500 million square kilometers of which about 30\% is land and 70\% is water, which brings us to the goals of this thesis.


### Thesis goals and outline
The motivation for this Master's thesis is to provide an answer to the question: What is the optimal resolution to detect human impact in satellite imagery, having in mind the economical cost of acquiring and processing the information? 
Determining this value is important not only for designing optimal satellite sensors but also to use optimal data sources when developing data-based remote sensing products. The goal here is not to build the top performance, state-of-the-art model to detect all sorts of human impact in satellite images, but rather to analyze the feasibility and cost of doing so at different resolutions. Of course, better algorithms could be trained on larger datasets to accurately identify certain types of human impact, but we consider a more general problem.

To address this problem, we divide the work into three parts. First, we develop a detector that is capable of identifying man-made structures on two aerial imagery datasets that we collect and annotate. Next, we study the performance of the detector in terms of classification accuracy as a function of image resolution i.e. the resolution per pixel. In the last part, we provide an approximate estimation of the costs of the entire pipeline. These also include metrics related to building, launching and maintaining a satellite. The estimation is performed for the entire spectrum of resolutions ranging from 0.03m to 16m.

The detailed outline of the thesis is listed below, and the Jupyter notebooks demonstrating the experimental work can be found on our Github repository (see \href{https://github.com/peterweber85/MasterThesis}{link} or reference \parencite{github}).

* Chapter~\ref{Chapter2} provides an overview of existing datasets and a detailed description of the construction of our own datasets. We further discuss the data manipulation pipeline.
	
* Chapter~\ref{Chapter3} gives an introduction to deep learning. We discuss theoretical concepts and recent advances in the field.
	
* Chapter~\ref{Chapter4} discusses the approach we followed to develop a detector capable of classifying man-made structures in aerial images.
	
* Chapter~\ref{Chapter5} presents the final results regarding the performance of the deep learning detector as a function of resolution for multiple image categories. It also provides an estimation of the cost to monitor the entire surface of the earth.
	
* Chapter~\ref{Chapter6} concludes the thesis and outlines potential next steps.	
	




## 2. Building datasets

In this chapter, we will give an overview of existing annotated aerial imagery datasets and outline the reasons why none of them is suitable for our investigation. Following this discussion, we will describe two approaches for obtaining our own labelled dataset.

### Requirements and considerations

Before we go into the presentation of labeled datasets we discuss the requirements that the dataset needs to fulfill in order to serve for the investigation in this thesis project. As a refresher, we want to detect human impact on aerial images and determine the dependency on resolution per pixel of a chosen evaluation metric. Ideally, the range for the resolutions should scale from a few tens of centimeters to a few tens of meters, whereas the images with low resolution can be generated from the high resolution images by downsampling. Having in mind previous arguments, we mainly need to consider three aspects. 

First, we need to have imagery data with labels that can be used to clearly distinguish between existing and non-existing human impact, respectively. This impact might be classified pixel wise, or as binary classification for the entire image, or as multi-class classification that can be translated into binary labelling. Second, we need a balanced dataset of approximately the same number of images for both classes, and a large variaty of different terrains within each class. Third, the images need to have a resolution per pixel which is equal or better than 1m. Also, the height and width of the images should measure at least $500\times500$ pixels, so that one has enough room for downsampling. 

### Existing datasets

In table \ref{table:datasets} we have summarized the most relevant remote sensing datasets with ground truth labels that can be found in literature. The datasets were collected using different publicly available data sources. These range from pure low-resolution satellite imagery (Sentinel-2) to high-resolution images taken with an aircraft (USGS) to a mix of different image sources (Google Earth). 

The satellite images have a resolution of equal or larger than 10~m and they are collected with the Sentinel-2 satellites of the European earth observation program Copernicus. Although the datasets from this source (BigEarthNet and EuroSat) are comparatively large, they do not suffice for our purpose, because the resolution is not good enough and the images are too small.

\begin{table}[h!]
	\begin{tabular}{l | l | l | l | l | l }
	name & source & images & resolution (m) & size (pixel) & categories \\
	\hline
	BigEarthNet \parencite{sumbul2019} & Sentinel-2 & 590,326 & 10, 20, 60 & 120, 60, 20 & $\sim$ 50 \\
	EuroSAT \parencite{helber2017}	& Sentinel-2 & 27,000  & 10 & 64  & 10 \\
	UCMerced \parencite{yang2010} & USGS & 2100 & 0.3 & 256 & 21 \\
	DeepSat \parencite{basu2015}  & USGS  & 405,000 & 1 & 28 & 6  \\
	AID \parencite{xia2016} & Google Earth & 10,000  & 0.5 - 8  & 600 & 30 \\
	PatternNet \parencite{zhou2017} & Google Earth & 30,400 & 0.06 - 4.69 & 256 & 38 \\
	\end{tabular}
	\caption{\textbf{Publicly available remote sensing datasets with labels.} The table  lists the name of the dataset together with the bibliographic reference. It also details the data source of the images. It contains a description about the number of images, the resolution of the images, the size of the square images in pixel, and the number of categories.}
	
\end{table}

The USGS National Map Urban Area Imagery collection \parencite{usgs} was utilized to collect remote sensing datasets in the two works UCMerced and DeepSat, where the former is the dataset that comes closest to our requirements. It features an image resolution of 0.3m per pixel, and the images have a height and width of 256 pixels. However, out of the 21 categories only 2 belong to images without human impact, while the other 19 show man-made structures. The DeepSat dataset unfortunately consists of image patches which are only $28 \times 28$ large, so that we aren't able to study these images as function of resolution.

The datasets using Google Earth as data source are collected using either the Google Earth or the Google Maps application programming interface (API). These images vary in resolution as well as in their original data provider since Google accesses several data sources. 
Both datasets, the AID and the PatternNet dataset, have about 30 categories with several hundred images in each category. Here, different categories have different pixel resolutions, and again most of the categories relate to urban areas so that we do not have sufficient images without human impact. Even the categories that in principle should not show human influence contain images that break this rule.

Overall, the main issue with these datasets stems from the fact that non of them was collected with the purpose to analyze the human footprint. Therefore they are very unbalanced, and do not contain sufficient variety of images for the classes without human influence. We hence decided to collect and label images by ourselves. In our first approach we used the Google Maps API, and in our final approach we used datasets from the USGS aerial imagery collection.

### Google Maps

Google has a public API that allows for querying images from their service Google Maps \parencite{google_maps_api}. In its most basic form, the API accepts as input parameters a lattitude (lat) and longitude (lon), a zoom, and the image size (in pixels). Given this set of parameters one can calculate the resolution in meter per pixel \parencite{gmaps_res_per_m}, which is given by
\begin{equation}
resolution \Big[\frac{meter}{pixel}\Big] = \frac{156543.03392 \cdot cos(\frac{lattitude \cdot \pi}{180})}{2 ^ {zoom}}.

\end{equation}

Equipped with this toolkit, we developed an automated image download pipeline that was based on one of several approaches of selecting and downloading images in a given area. In our first approach we selected images that were Gaussian distributed around a center location defined by a set of lat/lon coordinates. We further provided a precision parameter (standard deviation of the Gaussian), the number of images to download, a list of zooms, the desired image size, and the number of pixels to crop from the border of the image in order to remove e.g. the Google logo. Another approach consisted in downloading randomly sampled locations from within a rectangle defined by its upper left and lower right lat/long coordinates, respectively.

Although any of these two approaches would have served to build a complete dataset in an automated fashion, we finally decided to use a different data source due to the following reasons. According to our advisor from Satellogic, Google Maps images have one major drawback regarding the pixel resolution. The Google Maps image is an interpolation from different spectral bands, where the RGB color bands do not necessarily have the expected resolution. Therefore, the resolution estimated by Eq.~\ref{eq:gmaps_res_per_m} is not reliable for the three color channels. We did not further investigate into this issue and instead turned to a different solution, which is discussed next.

### USGS land cover

#### Getting the data

To be able to construct a balanced and representative dataset we were recommended to focus on images of the United States, 
because of the wealth of available high resolution aerial imagery data from USGS Earthexplorer \parencite{usgs}. A nice side effect of choosing the United States is that a large variety of images of different terrain and topology are available. We combined the aerial imagery datasets from USGS with additional information about land cover and land use from the USGS Land Cover Viewer \parencite{land_cover_viewer}, precisely to guarantee larger variety through the selection of data from distinct land use categories.

<img src = "../report/Figures/example_unproc.pdf" width = 800px>

For the determination of relevant geographic locations we excluded cities and highly developed urban areas, and instead focussed on unpopulated areas. Specifically, we limited our image search to the four land use categories agriculture, shrubland-grassland, semi-desert, and forest-woodland that can be found in the USGS Land Cover Viewer. Note that these categories served as a rough geographic orientation to pin down geolocations of interest. However not all the images could be assigned with absolute certainty to one unique category since we were not able to overlay both maps. Further, we selected many images from national parks because we found that it is significantly harder to find imagery data that does not show human influence. Whenever possible we also tried to collect images from both classes (man-made vs. natural) within a given area/terrain.

<img src = "../report/Figures/agriculture_sample.pdf" width = 800px>

<img src = "../report/Figures/shrubland-grassland_sample.pdf" width = 800px>

<img src = "../report/Figures/forest-woodland_sample.pdf" width = 800px>

<img src = "../report/Figures/semi-desert_sample.pdf" width = 800px>

Once an area was pointed out as a region of interest using the USGS Land Cove Viewer, we located it on USGS Earthexplorer and downloaded images from that area. In particular, we constructed two datasets with 0.3m and 1m resolution, respectively. The former was taken from the category High Resolution Orthoimagery and the latter from the category National Agriculture Imagery Program (NAIP). Note that the images in these categories usually have a height and a width of several thousand pixels, and hence occupy a few hundreds of Megabytes of disk space. We cropped smaller images out of the raw images, which will be discussed in more detail in the following section. Overall, we downloaded about 100 raw images for each dataset. An example is shown in Fig.~\ref{fig:example-unproc}.


#### Data processing and labeling

Our data processing pipeline consists of the following steps:

* Download large raw images.
* Crop images of size $512\times512$ pixel.
* Label images with either zero (no human impact), one (minimal human impact), two (clear human impact).
* Degrade images i.e. reduce number of pixels and thereby resolution per pixel.

Let us discuss each of these steps in more detail. An illustration of the first and second step of the image processing pipeline is given in Fig.~\ref{fig:example-unproc}. The white lines demonstrate the way we crop smaller images ($512\times512$ pixels) from the large raw image (in this case $5000\times5000$ pixels). We process all raw images in this manner, which yields approximately 80-150 processed images per raw image. We hence obtain about 10,000 processed images for each dataset. Within each category (USGS Land Cover categories) of the processed images we labeled a selected portion of the images by moving them into the folder with the respective label name. 

We have published our datasets via a Google Drive link \parencite{datasets}. The image folder of the published datasets contains the raw images, the processed images, and the labeled images. In this folder we follow a specific folder structure, which is shown below. Here pointy brackets (<parameter>) indicate a parameter and the content in the optional curly braces determines whether it is a folder pertaining to raw images. The first parameter is $pixels = 512$ and the second parameter represents the resolution of the dataset. Note that the label folders only exist in the case of processed images.
\vspace{10px}
\dirtree{%
	.1 \{raw-images-\}usgs-<pixels>-res<resolution>m.
	.2 semi-desert.
	.3 label-0.
	.3 label-1.
	.3 label-2.
	.2 agriculture.
	.3 label-2.
	.2 shrubland-grassland.
	.3 label-0.
	.3 label-1.
	.3 label-2.
	.2 semi-desert.
	.3 label-0.
	.3 label-1.
	.3 label-2.
}
\vspace{10px}
Annotating the images with labels was performed following certain rules. First, we classified images with no human impact at all into the class with label zero,  while we classified images with clear human influence into the class with label two. Ambigious images i.e. images with minimal human traces, such as a small walking path, were classified into class one. Second, we've put major effort into creating datasets that contain images of similar texture spread across all classes. If we for example classified a set of images of a certain forest type into class zero we classified another set of images with a similar forest type, but containing a building or a street, into the class two. We followed the latter rule for all categories except agriculture. The agriculture images all show human influence. We therefore classified them all with label two. By sticking to these rules, we are able to guarantee that the algorithm learns features that relate to the appearance of man-made structures, and not to image artefacts such as color or texture.

<img src = "../report/Figures/imstats.pdf" width = 800px>

In Figures \ref{fig:agriculture_sample} - \ref{fig:desert-sample} we display sample images for each of the four categories, repsectively. These images belong to the dataset that has a pixel resolution of 0.3m. The images from the 1m dataset have similar characteristics, but are not shown due to redundancy. Note that in Figs. \ref{fig:shrubland-sample} - \ref{fig:desert-sample} the first row represents images of label zero and the second row shows images that belong to label two. As mentioned above, the images in Fig.~\ref{fig:agriculture_sample} (agriculture) all contain human influence, and therefore belong to class two. 

The distribution of categories and labels is shown in Fig.~\ref{fig:imstats}. Overall, for the 0.3m dataset we classified about 2200 images, and for the 1m dataset we classified about 1450 images. Our main goal consisted in creating a balanced dataset between label zero and label two as can be seen from the distributions. A minority of images, roughly $10\%$ of all annotated images were assigned to label one. These images were used at random to investigate the behaviour of the Machine Learning classifier, which is discussed in chapter~\ref{Chapter5}.

<img src = "../report/Figures/demo_degrade.pdf" width = 800px>

The last step of the data processing pipeline consisted in downsampling the processed and labeled images, in order to obtain images with a lower resolution. We used a Lanczos filter \parencite{duchon1979} for the sampling, which is based on a sinusoidal kernel. In Fig.~\ref{fig:degrade} we show a few selected resolutions for an example image from the agriculture category. Note that here we only schematically depict an example in order to illustrate the process. However, in our Machine Learning pipeline the images are downsampled on the fly and the result of this process is not stored on disk (see Section \ref{sec:dl_architecture} for further details).

For this particular image one can observe how certain image features disappear as the image quality is decreased. Above a resolution of around 3m per pixel one is not able anymore to identify the building close to the right corner of the image. 
The texture of the track that leads up to the building is blurred above a resolution of around 4m per pixel. This shows how different elements in an image are not recognizable anymore once the resolution approaches their characteristic size.


## 3. Deep Learning

In this chapter, we will provide a short overview of the theoretical concepts and recent advances in the Deep Learning field. We will give a basic introduction to Neural Networks, and discuss convolutional Neural Networks in more detail as these are the type of algorithms utilized in this work. We further will summarize some of the most popular Convolutional Neural Network architectures.

### Introduction to Deep Learning

Deep Learning (DL) models have led to vast performance improvements in a large variety of domains, and therefore have gained substantial popularity over the last decades. These models were initially inspired by the human brain and analogies in neuroscience, which is why this class of algorithms was coined Neural Networks (NN). The two most popular Neural Network architectures are convolutional Neural Networks (CNN) and Recurrent Neural Networks (RNN). CNNs have driven major breakthroughs in visual object recognition \parencite{krizhevsky2012}, and image \parencite{zhang2015}, video \parencite{tompson2014} and audio \parencite{hinton2012} processing while RNNs brought about advances in research and applications on sequential data, i.e. in speech and text \parencite{collobert2011}. However, the superior performance of Neural Networks compared to traditional Machine Learning algorithms is not limited to the aforementioned domains. Other fields in which NNs have advanced the state-of-the-art include, for instance, bioinformatics \parencite{junshui2015} and the analysis of data from elementary particle physics \parencite{ciodaroc2012}.

Neural Networks define a class of models that are composed of a variable number of processing layers (Hidden units) of simple models, and are generally used to map a fixed size input (e.g. the pixels of an image) to a fixed size output (e.g. a category or a probability). A Hidden unit of a fully connected (FC) Neural Network has connections between all the nodes of the previous layer and the next layer. These connections are fully parametrized by the weights of the network. In analogy to a firing neuron, a non-linear activation function is applied to the output of the nodes of every Hidden unit. Historically, the \textit{sigmoid function}
$$\sigma(x) = 1/(1 + exp(-x))$$
and the \textit{hyperbolic tangent}
$$tanh(x) = 2\sigma(2x) - 1$$ 
have been used as activation functions. Nowadays, the most popular activation function is the \textit{rectified linear unit} (ReLU)
$$f(x) = max(0,x).$$
We will see the reason for this at the end of the section. 
In Fig.~\ref{fig:simple_neural_net}(a) we show an example of a fully connected feedforward 3-layer Neural Network. 

The strength of Neural Networks lies in their ability to learn arbitrarily complex non-linear input-output mappings \parencite{cybenko1989}, and that they can automatically extract features from raw data. The latter is in stark cotrast to traditional Machine Learning algorithms, which require careful feature engineering. For instance, when dealing with images, the multi layer architecture of Neural Networks allows to learn different features at every stage of the network, where the complexity and the abstraction of the learned features increases at every layer \parencite{farabet2013}.

<img src = "../report/Figures/simple_neural_net.pdf" width = 800px>

As all Machine Learning models, Deep Learning models are trained by minimizing an objective function i.e. by finding the optimal set of weights that achieve a specific input-output mapping. A typical objective function (also loss function) in a classification setting is the cross-entropy loss combined with a softmax. For the i-th training example it is
$$
L_i = -log\left(\frac{e^{f_{y_i}}}{\sum_j e^{f_j}}\right)
$$
where $f_{y_i}$ represents the class score computed for the real class, and $f_j$ is the j-th element of the vector of class scores. The total loss is the average over all $L_i$. The minimization of the objective function is accomplished by applying gradient descent based methods, in practice, often  stochastic gradient descent \parencite{bottou2008} or variants of it \parencite{ruder2016}. The computation of the gradient of the objective function with respect to all weights of the network is accomplished using backpropagation \parencite{rumelhart1986} (see Fig.~\ref{fig:simple_neural_net}(b)).
Intuitively, the backpropagation algorithm helps to quantify the influence of every weight of the network on the final error, so that one can decrease the error by updating the weights in the direction of the negative gradient.

Although artifical Neural Networks have been known and studied since the 1950s, it was only understood in the 1980s that mutlitlayer networks could be trained by backpropagation and stochastic gradient descent \parencite{lecun1989}. However, until recently, Neural Networks were still ignored by the computer-vision and speech-recognition communities, because of the believe that the objective function would get trapped in local minima. 

The advent of several new methods and technologies shall prove wrong the scepticism towards feasibly training deep Neural Networks. A requirement to reliable train large Neural Networks is the availability of large amounts of labelled data, as well as the necessary processing power. Both became available  about a decade ago with the emergence of "Big Data" and new powerful graphics processing units (GPUs).
Also theoretical advances helped to eliviate the difficulties to train Deep Learning models. These include the application of the ReLU non-linearity \parencite{glorot2011}, which solved the vanishing gradient problem, as well as the development of a particular type of NNs, the Convolutional Neural Networks. These networks are much easier to train than conventional FC networks, have less parameters, and they generalize better to unseen data. 

### Convolutional Neural Networks
Convolutional Neural Networks \parencite{vaillant1993, lecun1998} are specifically designed to process input data that has the shape of multiple arrays, such as the pixel values of a 2-dimensional image with three color channels. This is accomplished by using additional layers to preserve spatial structure. In general, a CNN is composed of several convolutional layers followed by a nonlinearity. These are often followed by a pooling layer, and a fully connected layer is used as the last layer of the network. The architecture of a small VGG convolutional net \parencite{simonyan2014}  used for classification is shown in Fig.~\ref{fig:convnet}.

With this design, CNNs take advantage of the natural properties of images. The central element here is the convolutional layer, which takes into account that local pixel values are highly correlated, and that the local statistics of images are invariant to translation \parencite{lawrence1997}. In particular, in a convolutional layer several small filters are slided spatially over the image computing dot products at every spatial location. The filters always extend the full depth of the input volume. For instance, a typical filter for a 3 color channel image might have dimensions $5 \times 5 \times 3$. Sliding this filter over an image of size, say  $32\times32\times3$, would lead to an activation map with dimensions $28\times28\times1$. 

The activation map is produced with one set of weights that belong to this particular filter. This concept is referred to as shared weights, which means that a comparatively small number of weights is shared across the entire image. As it is the case with conventional Neural Networks the weights, or parameters, are learned by applying gradient descent and backprogagation.  The number of parameters per filter is given by the spatial filter size, times the depth of the image plus a bias term. In the usual case of having multiple filters K the number of parameters is multiplied by K, which yields K activation maps. Each of these activation maps relates to one particular feature in the image where a spatial location in the activation map corresponds to the same spatial location in the input image (see Fig.~\ref{fig:convnet}(a)). 


<img src = "../report/Figures/convnet.png" width = 800px>
	

The dimensions of the output of every convolutional layer are controlled by three parameters: the stride S, zero-padding P, and the number of filters with size F. The stride is the interval at which the filter is slided over the image. Zero-padding has the porpuse to increase the image size by adding pixels with zero value at the border, so that the input and output dimensions can be matched (assuming stride is 1). For an input image of size W the output activation map will have
\begin{equation}
\frac{W - F +2P}{S} + 1
\end{equation}
pixels along every dimension. 

<img src = "../report/Figures/conv_pooling.pdf" width = 800px>
	

Pooling layers introduce coarse-graining in order to create invariance to small shifts and to decrease the number of parameters, which makes the representations smaller and more manageable. A pooling layer is applied to every activation map independently. It downsamples its input, in the most common case, by applying a max operator. This is shown schematically in Fig.~\ref{fig:conv_pooling}(b). Note that pooling layers don't have any parameters.

When stacking together mutliple combinations of convolutional layers followed by non-linearities and pooling layers, the filters learn a hierarchichal structure. The filters at earlier layers learn simple low-level features such as edges whereas the filters at later stages learn more complex high-level features, which are compositions of lower level features. In conclusion, the deeper a convolutional Neural Network is the more complex compositions of features it can learn.

### CNN architectures
The AlexNet was the first convolutional Neural Network that achieved remarkable results in the ImageNet classification task in 2012 (see Fig.~\ref{fig:imagenet}). It halfed the error in comparison to all competing non Deep Learning based approaches. In this competition deep Convolutional Networks were applied to a dataset with roughly 1 million images and 1000 classes. AlexNet was specifically designed to be trained on two GPUs with each 3GB of memory, which was sufficient to fit all the 60 million parameters inside. AlexNet had 8 layers, and used dropout as regularization technique.

<img src = "../report/Figures/imagenet_evolution.png" width = 800px>

The winner in 2013 was the ZFNet \parencite{zeiler2013}, which basically had improved hyperparameters compared to the AlexNet. In 2014, two networks were developed that were significantly deeper than previous networks. The VGG network \parencite{simonyan2014} had 19 layers and the GoogleNet had 22 layers \parencite{szegedy2014}. To be able to increase the number of layers, researchers from Oxford used very small filters ($3\times3$) in VGG thereby reducing the number of parameters per layer significantly. The GoogleNet first introduced the Inception module.The idea behind the Inception module is to have several small networks within the network that do multiple convolutions and pooling in parallel. In combination with getting rid of fully connected layers, the GoogleNet has only 5 million parameters, 12 times less than AlexNet. 

A significant improvement in the ImageNet challenge was achieved in 2015 by Kaiming He et al. \parencite{he2015}, when they submitted ResNet. During the development of this architecture the authors found that very deep networks perform worse than. Having excluded overfitting, they hypothesized that the origin of this observation must have it's roots in wrong optimisation of the objective function. The argument they gave was that a deeper network always must perform at least as good as a shallower network. This can be seen when one replaces some of the modules in the deeper network by identity mappings. Following this interpretation, they introduced residual blocks that contained an identity mapping in parallel to the convolutional layer. Having a so called skip connection, the network just needs to learn the residual denoted as $F(x)$ in Fig.~\ref{fig:resnet}. Taking advantage of this approach they were able to design networks with a depth of up to 152 layers, which allowed for halfing the error rate of the ILSVRC challenge in 2015. 

<img src = "../report/Figures/residual_modul.png" width = 800px>

<img src = "../report/Figures/resnet_architecture.png" width = 800px>

The architecture of a ResNet with 34 layers is shown in Fig.~\ref{fig:resnet}. The first convolutional layer has a filter size of $7\times7$ while all consecutive filters are chosen to be as small as possible ($3\times3$). After every convolutional block that contains mutliple convolutional layers and residual blocks the image is downsampled with stride 2 and the number of filters is doubled, so that effectively the spatial size decreases while the depth increases (and therefore the number of features that the network can learn). The network is terminated with an average pooling layer, and a 1000 fully connected layer for the 1000 classes of the ImageNet dataset. Overall, He and coworkers constructed ResNet architectures with 34, 50, 101, and 152 layers.

In recent years there have been developed many networks that go beyond ResNet. Some of them are extensions of ResNet (ResNext \parencite{xie2016}), or combinations of ResNet with other architectures (Inception-V4 \parencite{szegedy2016}), or  networks that do not make use of a residual block and instead use layer dropout (FractalNet \parencite{larsson2016}). Nevertheless, ResNet is still a state-of-the-art network, and that is why we have used it for the experiments in this work.

## 4. Proposed approach




In the previous chapters we have introduced the key components of the approach we followed in our study: on the one hand, we have described existing satellite image datasets and introduced the actual data we will consider, and on the other, we have discussed Deep Learning, how it works and how we can use it for our problem. Now, we are ready to describe our approach: the image feature extraction, the model architecture and the training scenario.

### Image features and transfer learning}

In order to train a model based on images, some sort of features need to be extracted. Traditionally, this image feature extraction was based on a set of hand-crafted detectors aimed to detect edges, corners, blobs and other feature descriptors. Some of these detectors are the Sobel filter, Laplacian of Gaussian (LoG), Difference of Gaussians (DoG), Determinant of Hessian (DoH), SIFT \parencite{Lowe1999,Lowe2004}, SURF \parencite{Bay2006}, Histograms of Oriented Gradients (HOG) \parencite{Dalal2005} and Gabor filters.


More recent approaches to image classification using Neural Networks have benefited from the existing and increasing computational power, and deep Convolutional Neural Networks have been able to achieve higher performance than traditional models. 

Yet, training a deep CNN from scratch for a particular problem requires a large and exhaustive dataset along with a huge amount of computational power. However, it has been shown that the architectures of pre-trained NN can be reused for other purposes and achieve equally great performance. This is known as \textbf{Transfer Learning}. Figure \ref{fig:transfer_learning_idea} schematizes this idea. These pre-trained architectures can be re-purposed by reusing the learned weights and either replacing the final layers of the net by some other classifier, or even fine-tuning all the layers for the specific problem. In any case, the initial layers of the Neural Network provide a great image feature extractor.

<img src = "../report/Figures/transfer_learning_idea.png" width = 800px>
	

In the next section we describe our approach using transfer learning from a ResNet architecture.

### Proposed architecture}

As described before (Sections \ref{sec:CNNArchitectures} and \ref{sec:transferLearning}), we can use for our problem a pre-trained ResNet with our own final classification layers. Hence, the architecture we propose for our problem consists of the activation layers of a ResNet, which act as the feature extractors of our images, followed by a shallow classifier made of a single dense (fully connected) layer. Figure \ref{fig:transfer_learning} gives a schema of this approach.

<img src = "../report/Figures/transfer_learning.png" width = 800px>
	

#### ResNet activations

The ResNet we consider (ResNet50) has a total of $49$ activation layers, so the output at each of them is different. Initial layers are able to recognize edges, textures and patterns while keeping an image size similar to the input. On the other hand, deeper activation layers show convolutions of higher order hierarchical structures. These structures are more complex and therefore the ResNet authors use much more channels in deeper layers. At the same time they decrease the spatial image size by applying stride 2 whenever they increase the number of filters. The purpose of the latter is to keep the number of parameters manageable.  

For instance, for an input image of (tensor) size $512 \times 512 \times 3$ (a $512x512$ image with $3$ RGB channels), the output of the first activation layer is of size $256 \times 256 \times 64$, the $10^{th}$ gives a $128 \times 128 \times 256$ tensor, and the last $49^{th}$ activation layer outputs $16 \times 16 \times 2048$. For our purpose, we will consider the final output of the ResNet ($49^{th}$ activation layer), although this could be further investigated and discussed.

Figures \ref{fig:act_agriculture} - \ref{fig:act_shrubland_grassland} shows 8 activation maps both for the $10^{th}$ and the $49^{th}$ layer for samples of different categories in the dataset. Some of the $10^{th}$ activations are particularly sensitive to edges, shadows, or textures, which later translate into more abstract outputs at the $49^{th}$ layer. Here one can observe that only a very selected number of neurons have fired, namely when a very specific feature was found on the corresponding position in the input image.

<img src = "../report/Figures/activations/agriculture_l2_s1_activation_10.png" width = 800px>
<img src = "../report/Figures/activations/agriculture_l2_s1_activation_49.png" width = 800px>
	

<img src = "../report/Figures/activations/forest-woodland_l0_s1_activation_10.png" width = 800px>
<img src = "../report/Figures/activations/forest-woodland_l0_s1_activation_49.png" width = 800px>
	

<img src = "../report/Figures/activations/semi-desert_l0_s1_activation_10.png" width = 800px>
<img src = "../report/Figures/activations/semi-desert_l0_s1_activation_49.png" width = 800px>
	

<img src = "../report/Figures/activations/shrubland-grassland_l0_s1_activation_10.png" width = 800px>
<img src = "../report/Figures/activations/shrubland-grassland_l0_s1_activation_49.png" width = 800px>
	


#### Complete architecture

For our purpose we considered the last ($49^{th}$) activation layer of the ResNet as the features of our images. These features can be extracted and saved on disk in order to speed up the process (as we did), or computed each time, and then passed through a simple fully connected Neural Network.

We terminated the ResNet architecture with a NN that consists of a single dense layer of $100$ or $200$ neurons with ReLU activation, followed by a single dense node with a Sigmoid activation acting as the classifier. This model is trained on the dataset with RMSprop optimizer \parencite{ruder2016} and a binary cross-entropy loss function. The same architecture is used and trained separately for each of the resolutions considered. 

This architecture (see Fig. \ref{fig:transfer_learning}) has been implemented with Python and Keras. Figure \ref{fig:model_keras} shows the model build, while in the following section we describe the complete training pipeline in more detail.

<img src = "../report/Figures/model_keras.png" width = 800px>
	


#### Training pipeline and experiments

As introduced in previous chapters, our goal with this model is to study how feasible it is (technically and costly speaking) to detect different kind of human impact on satellite images, and how this detection behaves for different image resolutions. To do so, we build two datasets of annotated images at base resolutions of $0.3m$ and $1m$ (see Chapter \ref{Chapter2}), which we later downgrade to a range of resolutions suitable for our study.

Starting from an image at its base resolution and size ($512\times512$ pixels), the downgrade process consists of downsampling (removing) pixels, so the image becomes smaller. Therefore, this imposes a limit on how far a given dataset can be downgraded, as the CNN ResNet model requires a minimum input size of $32\times32$ pixels due to the application of stride 2 at multiple layers \parencite{ResNetKeras}. Note that during the downsampling process the physical area displayed by the image is not changed. Tables \ref{tab:resolutions_03m} and \ref{tab:resolutions_1m} show the resolutions and sizes considered for the two datasets.


Next, for each of the datasets and downgraded resolutions we want to train a separate model and assess its performance. To this end, we consider the following pipeline for each of the datasets:


* Load the original images (at the original resolution) from disk along with the human impact labels and categories.
	
* Downsample the images to the desired resolution (from the lists in \ref{tab:resolutions_03m} and \ref{tab:resolutions_1m})
	
* Compute the ResNet activations (at the $49^{th}$ activation layer) of the resulting downgraded images. These activations can be saved to disk for later use if needed.
	
* Consider a stratified KFold split of the dataset (with $8$ splits) for cross-validation. That means, the dataset is split into $8$ sets with labels 0-1 equally distributed. Note that label 1 here represents the class with clear human impact, in contrast to the convention in chapter \ref{Chapter2} (where it was class 2).  Each cross-validation fold consists of around $300$ images for the $0.3m$ dataset, and around $200$ samples for the $1m$ dataset.
	
* Train the model (Fig. \ref{fig:model_keras}) separately for each combination of the $7$ training sets considering 30 epochs. The remaining set is used as a validation set to assess the accuracy. After training on all folds, the results of the $8$ experiments are averaged in order to obtain more consistent measures.
	
* Repeat the process for all downgraded resolutions.

Note that the splitting parameters of the cross-validation, the model complexity and the training epochs could be further analyzed in order to find the best combination for each of the resolutions tested. Actually, all the experiments consist of training NN models for two datasets, each with $16$ resolutions and $8$ folds per resolution, so every fine-tuning (like changing the size of the NN) implies several executions with some variability on each stage.

Nonetheless, as already mentioned in the introduction, the final goal of the experiments is to have a statistical reference of how well the models can be trained, and not to achieve the highest accuracy for each scenario. In order to do so, a larger dataset would be needed, with more well defined categories and objects, and a clear goal of what needs to be modeled.

The plots in Figures \ref{fig:conv_plots_03m} and \ref{fig:conv_plots_1m} (next pages) show the convergence of some of the models trained, for the different datasets and resolutions (base and lowest resolutions), and considering one of the cross-validation folds only. Also, two architectures for the dense layer have been tested: $100$ neurons and $200$ neurons. 

In general, the NN is able to converge and achieve a good accuracy (as shown in the plots), although for some particular splits of the data, it fails to converge and stays in a low accuracy point. This is probably due to the fact of having a relative small dataset. Hence, these particular folds are not going to be considered when computing the final averaged results.

In the next chapter we discuss in much more detail the results obtained from all these experiments.

<img src = "../report/Figures/results/convergence_plots_03m.png" width = 800px>
<img src = "../report/Figures/results/convergence_plots_1m.png" width = 800px>

## 5. Results

In this chapter we discuss the results obtained in this thesis. First, we analyze how the model works for a few given resolutions, so that we can be confident about its performance. Then, we consider how the accuracy changes with the resolution and within the categories. Finally, we discuss the cost around Satellite imagery to analyze once the entire earth. 

### Transfer learning on aerial imagery

The first thing we want to investigate from our experiments is whether the approach followed is able to achieve good results. That is, we want to evaluate if using a pre-trained ResNet as a feature extractor for aerial images allows the trained model to properly discriminate the existence of human impact. 

Tables \ref{tab:Results_03m_100n} - \ref{tab:Results_1m_200n} in Appendix \ref{Tables} show that the accuracies obtained with all the experiments are indeed remarkable. All these results are discussed in more detail in the next section, but let us begin by focusing on few cases of the $0.3m$ dataset in order to understand how the models are behaving. In table \ref{tab:Results_03m_100n} we can see that an accuracy of around $0.9226$ is achieved on the base resolution ($0.3m$), while it drops to $0.8690$ of the last downgraded resolution of the dataset ($4.8m$).

Let us consider first some examples of correctly and wrongly classified images at the base resolution (for one of the cross-validation folds), which are shown in Figures \ref{fig:dataset03m_res03_correct} and \ref{fig:dataset03m_res03_wrong} respectively. The first set of samples shows that the model accurately detects clear human impact related to agriculture (2nd picture in the second row) and paths. On the other hand, the second set shows that it might fail to detect it when the impact is subtle, covering a small region of the image, or when it can even be confused with natural structures (or vice versa).

Note that, from this point, \textit{label 1} refers to images with clear human impact, which were defined as \textit{label 2} when building the datasets in Chapter \ref{Chapter2}.

<img src = "../report/Figures/results/class_dataset03m_res03_correct.png" width = 800px>

<img src = "../report/Figures/results/class_dataset03m_res03_wrong.png" width = 800px>

The same kind of analysis can be done for the last resolution, $4.8m$. Figures \ref{fig:dataset03m_res48_correct} and \ref{fig:dataset03m_res48_wrong} show correctly and wrongly classified images at this resolution. The first set shows that the model detects human impact when it is still evident, even with the low resolution, but the second set indicates that it commits mistakes when the human impact evidences are lost with the downgrade process. Similarly, it might classify as man-made structures patterns that are indeed natural.

<img src = "../report/Figures/results/class_dataset03m_res48_correct.png" width = 800px>

<img src = "../report/Figures/results/class_dataset03m_res48_wrong.png" width = 800px>

This can also be seen with Figure \ref{fig:dataset03m_res03_res48_comp}, which shows images that are correctly classified at $0.3m$ resolution (top row) but wrongly classified at $4.8m$. The first and third pair of images demonstrate that, when the human impact is subtle, the model missed it in the downgraded resolution. Conversely, non human activity can also be misclassified at lower resolutions (second and fourth images).

<img src = "../report/Figures/results/class_dataset03m_res03_comp_correct.png" width = 800px>
<img src = "../report/Figures/results/class_dataset03m_res48_comp_wrong.png" width = 800px>

Finally, we can investigate how the model behaves with images where human impact is very subtle. For this purpose, we consider the images with the intermediate label (\textit{label 1} in Chapter \ref{Chapter2}, Figure \ref{fig:imstats}). The model has never been faced with these images, so this can give a good perception of whether the models have been able to learn relevant features of human impact. Figure \ref{fig:dataset03m_res03_l1} shows some of these images with the label predicted by the model in the title. Even if man-made structures in these pictures are small, the model is able to detect straight lines and shapes as human activity. 

<img src = "../report/Figures/results/class_dataset03m_res03_l1.png" width = 800px>

All in all, we can conclude that the model is able to achieve a great accuracy. It is able to generalize to unseen, subtle images, and still be accurate for lower resolutions, were the amount of pixels and information per images become much smaller. 

### Man-made structures detection at different scale

Now that we know that the trained models are able to detect, with high accuracy, human impact from our datasets, we are ready to analyze the results for different resolutions. From all the experiments (datasets, architectures, resolutions and cross-validations), the results have been stored and aggregated. As mentioned in the previous chapter, the models were not able to converge for some particular splits of the datasets. Hence, these few folds have been ignored when aggregating the results. Tables \ref{tab:Results_03m_100n} - \ref{tab:Results_1m_200n} in Appendix \ref{Tables} summarize the accuracies obtained for each of the datasets ($0.3m$ and $1m$) and downgraded resolutions, both the overall accuracy and by category. 

Similarly, Figure \ref{fig:acc_res_03m_1m} shows the overall accuracies obtained for all resolutions. From this we can see that similar accuracies are achieved for both datasets on the same degraded resolution, which means that both datasets are comparable and can be considered together. Also, we realize that increasing the size of the model from $100$ neurons to $200$ does not have a big impact on the accuracy, but tends to perform slightly better. Hence, for other results later we will focus on this architecture only.

<img src = "../report/Figures/results/acc_res_03m_1m.png" width = 800px>

On closer inspection, we also detect that the accuracy on the base resolution ($0.3m$ and $1m$) is always slightly worse than the next resolution. This is probably because the input image size at the base resolution is quite large ($512\times512$), which makes the dense layer much more complex to train. Indeed, Figures \ref{fig:conv_plots_03m} and \ref{fig:conv_plots_1m} in Chapter \ref{Chapter4} already suggest that the models struggle to be optimized. More sample images would be required in order to compensate for the complexity and achieve a greater accuracy. 

Finally, the overall conclusion from this plot is that, as expected, better resolutions (less than $2m$) allow for greater accuracies, of over $90\%$, which means a great success considering that the images in the datasets are balanced between having or not human impact. Furthermore, accuracy is still good for resolutions between $2m$ and $8m$, staying between $85\%$ and $90\%$. From $8m$ onwards, accuracy drops to $80\%$ and the model is not able to detect more subtle elements of man-made structures.

Now let us consider how these accuracies behave for each of the land use categories in the datasets. As discussed in section \ref{usgs_data}, these categories are rough approximations of the kind of terrain and human impact, with images that could be exchanged between categories, but overall these can give and idea of the accuracies when analyzing different kind of terrains. Indeed, Figures \ref{fig:acc_all_cat_03m} and \ref{fig:acc_by_cat_03m} show that, for the $0.3m$ dataset (and $200$ neurons model), accuracy differs substantially between categories. Fig. \ref{fig:acc_all_cat_03m} shows the global comparison between categories, while Fig. \ref{fig:acc_by_cat_03m} allows for a better comparison of each category with respect to the overall accuracy. 

<img src = "../report/Figures/results/acc_res_all_categories_03m.png" width = 800px>

<img src = "../report/Figures/results/acc_res_by_category_03m.png" width = 800px>
	

These plots have been obtained once the model was trained for all the categories. Then, the accuracy on the validation set was computed for each individual category and over all images in the set. The validation set in each iteration of the cross-validation was small, consisting of few hundred images, and a homogeneous representation of the categories was not imposed (validation samples were picked randomly, only preserving proportion between having or not human impact), which explains the large variability for each experiment (vertical lines in the plots).

Although the category is not taken into account when training, we can see that the models are capable of detecting agriculture-related human impact with high accuracy (over $95\%$), without really being affected by the drop in accuracy. Shrubland-grassland category also has a good accuracy, while the model performs worse in semi-desert and forest-woodland images. That means that the models are able to capture textures and patterns related to agriculture quite easily, while the other categories have more variable features.

Similar results are obtained for the $1m$ dataset, which are shown in Figures \ref{fig:acc_all_cat_1m} and \ref{fig:acc_by_cat_1m}. The models are able to achieve a great accuracy when detecting agriculture-related human impact (over $95\%$), independently of the resolution considered, but the performance drops for the other 3 categories. The biggest drop is observed at $8m$ consistently over the three non-agriculture categories. 

<img src = "../report/Figures/results/acc_res_all_categories_1m.png" width = 800px>

<img src = "../report/Figures/results/acc_res_by_category_1m.png" width = 800px>
	


### Cost estimation

We discuss the financial cost associated to building and launching a satellite, and to renting infrastructure for performing the entire image processing pipeline. We further study the cost as a function of pixel resolution. However, our estimates are very rough approximations because many factors are involved and large variations occur between them. To give an example, choosing one material over the other might change the cost of manufacturing and launching a satellite by one order of magnitude. It is also completely different to have a satellite for 3 years in space, or to target a lifespan of 20 years. 

Having this in mind, we follow laws from physics to estimate the dependency of the satellite cost on resolution. First, the cost of launching a satellite into the orbit scales linearly with its mass, which is given by the amount of fuel needed. Second, the mass of the satellite scales quadratic with resolution so that overall we obtain a cubic dependency of financial cost on resolution. The latter increase in cost is associated with the optical instruments used. As a reference for the satellite cost we use a Skysat satellite from Planet \parencite{skysat_planet} that has a resolution of about 1m and a value of $\$30$ million. This amount was provided to us by Satellogic and includes construction, launch and maintenance during the satellite's lifespan.

Our final goal is to give an estimation of the expenditure to monitor once the entire surface of the earth (about 149 million km$^2$). To this end, we multiply the satellite cost by the ratio: time needed to scan the earth over the satellite's lifespan. Further, a satellite can map 1 million km$^2$ at 1m resolution in 4.2 days \parencite{satellogic_youtube}. We hence can calculate the satellite cost per km$^2$. With $area~per~lifespan = 10^6 \times \frac{10\cdot365}{4.2}~km^2$ we obtain $cost~satellite~per~km^2 = cost~satellite/area \approx 0.035~\$/km^2$.

\begin{table}[h!]
	\begin{tabular}{l | l | l | l | l}
		description & cost & unit & cost (\$/km$^2$) & cost (\$/pixel) \\
		\hline
		process raw data & & & 0.004 & $4 \times 10^{-9}$ \\
		hot storage  & $72\times 10^{-6}$ & \$/(km$^2$/month) & 0.000864 & $8.64\times10^{-10}$ \\
		cold storage  & $36\times 10^{-6}$ & \$/(km$^2$/month) & 0.000432 & $4.32\times10^{-10}$ \\
		archive storage  & $9\times 10^{-6}$ & \$/(km$^2$/month) & 0.000108 & $1.08\times10^{-10}$ \\
		download data & 8 & \$/Gb & 0.021 & $2.1  \times 10^{-8}$\\
		serving to final client & 0.09 & \$/Gb & 0.000236 & $4.7232 \times 10^{-10}$\\
		prediction (AWS) & 0.05 \& $\sim$6 & \$/h \& s/km$^2$ & 0.00145 & $1.45 \times 10^{-9}$\\
	\end{tabular}
	\caption{\textbf{Costs for image data processing.} All costs except the prediction are provided by Satellogic.}
	
\end{table}

Another cost intensive block when capturing satellite imagery involves image data processing for which the cost scales quadratic with resolution. For example, an operation that costs 100\$/km$2$ at 1m resolution will cost only 1\$/km$^2$ at 10m resolution. The data processing step consists of multiple parts: transformation of raw data into image pixels, storing data in a hot, cold, and archive storage, downloading data from the satellite, serving it to the final client, and in our case predicting human impact. These costs are summarized for 1m resolution in table \ref{table:data_costs}. Note that we used the conversion factor 0.002624 for an image to convert from Gigabytes to km$^2$ (2$\times$ compressed) and we assume 12 months of data storage. The prediction step is estimated by loading 4 images that each have an area of about $500\times500m^2$, calculating the ResNet activations of the final layer, and predicting the class using the models trained in chapter~\ref{Chapter4} in an ensemble fashion. This part amounts to a processing time of 6s for an area of 1km$^2$, which can be converted into costs per km$^2$ assuming 0.05\$/h of AWS EC2 compute~\parencite{aws}.

To finally obtain the dependence of the resolution on the total financial cost we sum the data cost per km$^2$ and the satellite cost per km$^2$ at 1m resolution, and convert to cost per pixel ($\times 10^{-6}$). We then multiply with the number of pixels necessary to cover the entire surface of the earth. Here the satellite cost per km$^2$ is a cubic function and the earth surface in pixel is a quadratic function in resolution. The result is shown in Fig.~\ref{fig:costs}. We obtain a cost of about \$15 million dollars at 1m resolution with a very step slope towards better resolutions. At 0.3m resolution the cost is two orders of magnitude higher than at 1m while for worse resolutions the cost decreases by two orders of magninute when the cost is a factor 10 larger. We conclude that for worse resolutions the data processing cost is the dominating cost whereas for very good resolutions the satellite cost dominates.

<img src = "../report/Figures/costs.pdf" width = 800px>
	

## 6. Conclusions

In this final chapter we discuss and close the different aspects of the project, from the problem definition itself, to the datasets build and models developed, and we conclude our thesis with some further work ideas to continue and enhance our approach.

### The Problem

The intial phase during the development of the project consisted on clearly defining the problem to study. The goal was to investigate which satellite imagery resolutions allowed for an accurate detection of man-made structures, and what would be the cost associated. For that, we needed to define the scope of human activity to consider, look for suitable datasets for this study (which eventually lead to building our own datasets), and define the actual technical problem to be modeled, in order to evaluate the accuracy by resolution. 

Both for the datasets and the problem, we needed them to be feasible enough to not require high technical and computational efforts (which would be, for instance, trying to detect every type of human activity in the images, providing their position and shape, and classifying them into several more categories). On the other hand, we needed it to be a realistic situation, so the results obtained could be extrapolated to other, more complex scenarios.

All in all, having a well-defined problem scope and a good approach to tackle it allowed us to achieve remarkable and realistic results.

### The Datasets

After investigating existing datasets of labeled satellite images, we could not find the one suitable for our purpose, as most of them were mainly focused towards urban areas, or were just build for some other different goal that would not work for us. Hence, we decided to build it ourselves. It had to be representative enough to pose a challenge for our models, yet feasible to be build with our available time and resources. The four categories considered (agriculture, shrubland-grassland, semi-desert, and forest-woodland) and the balance between non-existent and existent human impact images allowed us to build a good and representative dataset, which makes reasonable to eventually generalize the results obtained to other scenarios.

Of course, we acknowledge that having a larger dataset of images, annotated with higher degree of detail, like position, shapes and types of man-made or natural structures, would be great to build high performance models, capable of detecting all sorts of human impact with far better accuracy. Nevertheless, this high-precision goal could not be fitted into our general purpose.

### The Models and Results

Using a pre-trained CNN like the ResNet helped us to speed up the training process and achieve good results without requiring a very large dataset and computational power. The binary classification problem considered turned out to be feasible and representative of how accuracy is affected with a decrease in the image resolution. 

From the results we observe that, as expected, the higher the resolution the better, but also that there seems to be sweet spot between $1$-$2m$ and $8m$ where, except for the more subtle forms of human activity, most of the man-made structures studied are detectable with good accuracy. This trade-off with the resolution allows to consider more cost-economic satellite solutions without dramatically compromising accuracy and utility. For instance, operating a satellite at $2m$ (or $8m$) instead of $1m$ reduces the cost approximately by a factor 6 (or $100$).

### Further work

With all that said, we realize that there is still plenty of space for further work and investigations, so let us now indicate some of these ideas.

First of all, having a better dataset could help improving the investigations and opening new lines to explore. It could be improved and enlarged with more variate images, with a better and more consistent classification, and including more detailed annotations of the position and type of objects or structures appearing. This would allow to train more accurate models capable of detecting all these kind of human impact.

Regarding the model, other techniques for feature extraction could be studied, like other pre-trained Neural Networks, and the parameters itself (like the number of activations to consider, the architecture of the model or the training phase) could be further fine-tuned. Additionally, the pipeline could be made more robust so that it could ingest a larger amount of data, as part of the improvements suggested for the dataset. And, of course, having a powerful computational cluster would allow to speed up the processes and target more ambitious goals.

A more in depth study of the results could help to understand more precisely on which images the algorithms fail, what kind of information are learning (patterns, colors, shapes, etc) and how to enhance them.

Finally, it would be interesting to have a more detailed analysis of the costs associated to all this solution, from data gathering, processing and modeling to the production implementation itself. Also, taking into account other related factors, like infrastructure needed, legal aspects and ecological footprint \parencite{Strubell2019} would give a more complete idea of the viability of global satellite image analysis.

In conclusion, this project has allowed us to investigate a relatively new topic, covering from the data gathering to the technical implementation using state-of-the-art tools, and leaving the door open to further investigations.

## A. Files and Code

All the files used in this project, including the image datasets build and code generated, are available online:

* The images datasets are published in Google Drive (see \href{https://drive.google.com/open?id=1Hjod1ZTuSIW3VNO2IuGoq_iagI3imnJQ}{link} or reference \parencite{datasets}).
	
* All the code produced and used in these analysis is available in a GitHub repository (see \href{https://github.com/peterweber85/MasterThesis}{link} or reference \parencite{github}). It includes Python libraries generated, scripts and Jupyter Notebooks.


